# Next overpass predictor

## Background
* Knowing the time of a satellite overpass (OP) at a precise location is crucial to plan and prepare disaster impact studies. 
The script below can be used to predict the overpasses of the Landsat 8 & 9 and Sentinel 1 & 2 satellites over a  selected location. For Landsat 8 this occurs every 16 days and for Sentinel 2A / 2B this occurs every 10 days.

* The code calls the Python package 'next_pass' located at https://github.com/ehavazli/next_pass. The latter predicts the next overpass of the satellite of interest by scanning the relevant acquisition plans:
	- Landsat acquisition plans (json files) : https://landsat.usgs.gov/sites/default/files/landsat_acq/assets/json/cycles_full.json
	- Sentinel acquisition plans (KML files to import to Google Earth Pro) : https://sentinel.esa.int/web/sentinel/copernicus/sentinel-1/acquisition-plans  

## Description

All what a user needs to provide is the precise location for which he desires to identify the next overpasses. The location can be inputted as (latitude, longitude) or as the name of the city of interest. The script returns the next collect for Sentinel-1 and Sentinel-2 and the next passes, in ascending and descending directions separately, for Landsat-8 and Landsat-9:

- Specify a location 
- Run find_next_overpass for Sentinel-1, Sentinel-2 and the Landsats (8&9) 
- Visualize each of the above predicted overpass 

The outputs of next_pass can be compared against overpasses of the site you are interested in using the ESA Orbital Prediction and Overpass Tool (OPOT) at https://evdc.esa.int/orbit/ 


## Getting started
To run the overpass predictor with the given location, run all cells in the notebook starting with the "Load packages" cell.

### Load packages and functions

In [ ]:
import next_pass
import utils
import folium
import re  # Import regular expressions
import random  # To generate random colors
import earthaccess
import leafmap
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape, box
from geopy.geocoders import Nominatim
from argparse import Namespace
from datetime import datetime

In [ ]:
import colorsys

# Style function for the bounding box GeoJSON layer
def style_function(feature):
    return {
        'fillColor': '#808080',  # Gray fill color
        'color': '#000000',       # Black border color
        'weight': 4,              # Thicker border (increased thickness)
        'fillOpacity': 0.3        # Fill opacity (adjust if needed)
    }
# Function to generate random hex color
def random_color():
    return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Function to print text with color in console (ANSI escape code)
def print_colored_text(text, color):
    # Escape sequence for colored text
    print(f"\033[38;2;{color[0]};{color[1]};{color[2]}m{text}\033[39m")

# 
def hsl_distinct_colors(n):
    colors = []
    for i in range(n):
        # Generate colors with different hues
        hue = i / float(n)  # Hue ranges from 0 to 1
        color = colorsys.hsv_to_rgb(hue, 1.0, 1.0)  # Convert HSL to RGB
        # Convert from RGB (0-1) to hex (#RRGGBB)
        rgb = [int(c * 255) for c in color]
        hex_color = "#{:02x}{:02x}{:02x}".format(*rgb)
        colors.append(hex_color)
    return colors

def spread_rgb_colors(n):
    colors = []
    step = 255 // n  # Divide the color space into n parts
    for i in range(n):
        # Spread out the color values across the RGB spectrum
        r = (i * step) % 256
        g = ((i + 1) * step) % 256
        b = ((i + 2) * step) % 256
        hex_color = "#{:02x}{:02x}{:02x}".format(r, g, b)
        colors.append(hex_color)
    return colors

def hsl_distinct_colors_improved(num_colors):
    colors = []
    
    for i in range(num_colors):
        # Set Hue (H) to a random value, excluding extremes like 0° (red) and 60° (yellow)
        hue = (i * 360 / num_colors) % 360
        
        # Set Saturation (S) to a high value (e.g., 70%) for vivid colors
        saturation = random.randint(60, 80)  # Avoid dull colors
        
        # Set Lightness (L) to a lower value to avoid bright, light colors like yellow (range 30-50%)
        lightness = random.randint(30, 50)  # Darker or neutral colors

        # Convert HSL to RGB using the colorsys library
        r, g, b = colorsys.hls_to_rgb(hue / 360, lightness / 100, saturation / 100)

        # Convert RGB to hex format (RGB values are in [0, 1], so multiply by 255)
        hex_color = "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))
        colors.append(hex_color)
    
    return colors

### Specify location
Start with selecting the location by  precising the latitude/longitude. 

In [ ]:
# Locations (here La Crescenta). If overpasses are sought for 1 location, just set min and max params to the same value
lat_S, lat_N, lon_W, lon_E = 34.230429, 34.230429, -118.2350733, -118.2350733

In [ ]:
# Users can provide SNEW coordinates to define a bounding box
lon_W, lat_S, lon_E, lat_N = 67.982054, 26.198739, 68.543065, 28.568858

In [ ]:
# Users can also use a predefined polygon (kml file) exported from Google Earth, for example
location_file_path = '/Users/ifenni/Desktop/JPL/ARIA_OPERA/data/KML/MiltonHurricaneFlorida.kml'

### Specify satellites of interest 
For now, the tool operates for Sentinel 1A and 2A and Landsat 8 and 9.

In [ ]:
# Satellites
sat1 = "sentinel-1"
sat2 = "sentinel-2"
sat3 = "landsat"

In [ ]:
# Create the args object to pass to find_next_overpass
args = Namespace(
    bbox=(lat_S, lat_N, lon_W, lon_E),  # assuming lat_min, lat_max, lon_min, lon_max are defined
    aoi_file=location_file_path,  # assuming locationfile_path is defined
    satellite=sat1  # assuming sat1 is defined (e.g., 'sentinel-1', 'sentinel-2', 'landsat')
)

### Run next_pass
use next_pass to predict the overpasses of the above satellites over the selected location

In [ ]:
print("*** ",sat1," ***")
result1 = next_pass.find_next_overpass(args)
# result1 is a dictionary 
s1_next_collect_info = result1.get("next_collect_info", "No collection info available")
s1_next_collect_geometry = result1.get("next_collect_geometry", None)
print(s1_next_collect_info)

In [ ]:
print("*** ",sat2," ***")
args.satellite = sat2
result2 = next_pass.find_next_overpass(args)
s2_next_collect_info = result2.get("next_collect_info", "No collection info available")
s2_next_collect_geometry = result2.get("next_collect_geometry", None)
print(s2_next_collect_info)

In [ ]:
print("*** ",sat3," ***")
args.satellite = sat3
result3 = next_pass.find_next_overpass(args)

### Overpasses Vizualisation  
The below vizualization tool shows the path of a selected satellite at the predicted date/time

In [ ]:
# Start by choosing what satellite to visualize 
sat_to_visualize = 'Sentinel-1'; # can be Sentinel-1 or Sentinel-2

if location_file_path:
    area_polygon = utils.create_polygon_from_kml(location_file_path)
    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': [area_polygon]}, crs="EPSG:4326")  # WGS 84 CRS
    # Create a Folium map centered at the bounding box centroid
    m = folium.Map(location=[area_polygon.centroid.y, area_polygon.centroid.x], zoom_start=4)
    # Add the bounding box as a GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="Area of interest", style_function=style_function).add_to(m)
elif lat_S == lat_N and lon_W == lon_E:
    # Create the point
    point = Point(lon_E, lat_N)

    # Create a Folium map centered at the point location
    m = folium.Map(location=[point.y, point.x], zoom_start=4)

    # Add a cross-shaped marker to the map
    folium.Marker(
        location=[point.y, point.x],  # Latitude, Longitude
        icon=folium.Icon(icon='glyphicon-remove', icon_color='red', prefix='glyphicon')  # Cross symbol with red color
    ).add_to(m)
else:
    # Create the bounding box as a polygon
    bounding_box = box(lon_W, lat_S, lon_E, lat_N)

    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs="EPSG:4326")  # WGS 84 CRS

    # Create a Folium map centered at the bounding box centroid
    m = folium.Map(location=[bounding_box.centroid.y, bounding_box.centroid.x], zoom_start=4)
    # Add the bounding box as a GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="Bounding Box", style_function=style_function).add_to(m)

if sat_to_visualize == 'Sentinel-1':
    vi_next_collect_info = s1_next_collect_info
    vi_next_collect_geometry = s1_next_collect_geometry
elif sat_to_visualize == 'Sentinel-2':
    vi_next_collect_info = s2_next_collect_info
    vi_next_collect_geometry = s2_next_collect_geometry
else:
    vi_next_collect_info = l8_next_collect_info
    vi_next_collect_geometry = l8_next_collect_geometry
        
print('\n ** Visualizing overpasses for ',sat_to_visualize,' ** \n')
# Add each Polygon in next_collect_geometry
lines = vi_next_collect_info.split("\n")
# Clean lines by keeping only those that contain numbers (1-9)
cleaned_info = [line for line in lines if re.search(r'[1-9]', line)]  # Line must contain digits (1-9)
vi_next_collect_info_list = cleaned_info  # Now it's a list of strings (one per row in the table)
num_polygons = len(vi_next_collect_geometry)
num_info_lines = len(vi_next_collect_info_list)
#print(num_polygons)
#print(num_info_lines)

# Use the HSL distinct colors function
distinct_colors_list_1 = spread_rgb_colors(num_polygons)
distinct_colors_list_2 = hsl_distinct_colors(num_polygons)
distinct_colors_list_3 = hsl_distinct_colors_improved(num_polygons)

if vi_next_collect_geometry:
    for i, (polygon, info) in enumerate(zip(vi_next_collect_geometry, vi_next_collect_info_list), start=1):
    #i=3 
    #polygon =vi_next_collect_geometry[2]
    #info = vi_next_collect_info_list[2]
    
        if isinstance(polygon, Polygon):  # Ensure it's a valid Polygon
            # Get a distinct color for each polygon
            color = distinct_colors_list_3[i - 1]
    
            # Print the info with corresponding color in the console
            print_colored_text(f"{info}", tuple(int(color[i:i+2], 16) for i in (1, 3, 5)))
    
            
            geojson_data = gpd.GeoSeries([polygon]).__geo_interface__
            folium.GeoJson(
                geojson_data, 
                name="Next Collect Area",
                style_function=lambda x, color=color: {"color": color, "weight": 2, "fillOpacity": 0.3},
                popup=folium.Popup(f"Polygon: {info}", max_width=300)  # Display corresponding info line
            ).add_to(m)

print('')
# Display the map and save to file
m.save("bounding_box_map.html")
m  # If using Jupyter Notebook

# Available OPERA Products

## Description
* Here we will use Leafmap and Earthaccess to explore OPERA DSWx Products.
* The Leafmap library provides a suite of tools for interactive mapping and visualization in Jupyter Notebooks. Leafmap version 0.30.0 and later offers tools specifically for accessing NASA Earthdata by building on the newly developed NASA Earthaccess library. Earthaccess provides streamlined access to NASA Earthdata and simplifies the authentication and querying process over previously developed approaches. 
* This section is designed to leverage tools within Earthaccess and Leafmap to facilitate easier access and visualization of OPERA data products for a user-specified area of interest (AOI). 

## OPERA DSWx Products 

The Dynamic Surface Water eXtent (DSWx) products map pixel-wise surface water detections using optical or SAR imagery. The DSWx suite is composed of complementary products, which are named according to their input datasets. In the present section, we will focus on: 

- DSWx from Harmonized Landsat Sentinel-2 (DSWx-HLS)
- DSWx from Sentinel-1 (DSWx-S1)


## View the available OPERA products
Note above that the `earthdata_df` contains a number of columns with metadata about each available product. the `ShortName` column will be used to produce a new dataframe containing only OPERA products. Let's view the available products and their metadata.

In [ ]:
### View Earthdata datasets
earthdata_url = 'https://github.com/opengeos/NASA-Earth-Data/raw/main/nasa_earth_data.tsv'
earthdata_df = pd.read_csv(earthdata_url, sep='\t')
opera_df = earthdata_df[earthdata_df['ShortName'].str.contains('OPERA', case=False)]
### Print the available OPERA datasets 
print('Available OPERA datasets:', opera_df['ShortName'].values)

## Define an area of interest (AOI) and time period of interest (TOI)
* The script below will use the area of interest (AOI) indicated above to predict overpasses. 
* To retrieve OPERA products available for this AOI, the user should define a time interval of interest (TOI).  

In [ ]:
### This cell initializes the AOI and TOI.
if location_file_path:
    AOI = area_polygon.bounds
else:
    # Create the bounding box as a polygon
    AOI = (lon_W, lat_S,lon_E, lat_N)
print(AOI)
c_lat = (AOI[1] + AOI[3]) / 2  # (miny + maxy) / 2
c_lon = (AOI[0] + AOI[2]) / 2  # (minx + maxx) / 2

#Here we have selected two dates. This could expand to include date ranges but then image mosaic rules should be considered (not included here)
StartDate_PreEvent="2024-10-03T00:00:00"  #Pre-event image start date
EndDate_PreEvent="2024-10-03T23:59:59"    #Pre-event image end date

StartDate_SynEvent="2024-10-11T00:00:00"  #Syn-event image start date
EndDate_SynEvent="2024-10-11T23:59:59"    #Syn-event image end date

### Query the OPERA DSWx-HLS dataset for the AOI

In [ ]:
dswx_results_PreEvent, dswx_gdf_PreEvent = leafmap.nasa_data_search(
    short_name='OPERA_L3_DSWX-HLS_V1',
    cloud_hosted=True,
    bounding_box= AOI,
    temporal=(StartDate_PreEvent, EndDate_PreEvent),
    count=-1,  # use -1 to return all datasets
    return_gdf=True,
)

dswx_results_SynEvent, dswx_gdf_SynEvent = leafmap.nasa_data_search(
    short_name='OPERA_L3_DSWX-HLS_V1',
    cloud_hosted=True,
    bounding_box= AOI,
    temporal=(StartDate_SynEvent, EndDate_SynEvent),
    count=-1,  # use -1 to return all datasets
    return_gdf=True,
)

In [ ]:
dswx_results_PreEvent[0] #Note this just shows a single MGRS/HLS tile


In [ ]:
dswx_results_SynEvent[0] #Note this just shows a single MGRS/HLS tile

### View the DSWx-HLS metadata and footprints

In [ ]:
dswx_gdf_PreEvent.head()

In [ ]:
### Plot the location of the tiles 
#dswx_gdf_PreFlood.explore(fill=False)
m_HLS = leafmap.Map(center=(c_lat, c_lon), zoom=5)
style = { "color": "blue",        # Line color
        "weight": 2,            # Line thickness
        "fillOpacity": 0.0      # Transparent fill
        }
m_HLS.add_gdf(dswx_gdf_PreEvent, layer_name="DSWX Pre-Event", style=style)
m_HLS

In [ ]:
### Plot the location of the tiles 
#dswx_gdf_SynEvent.explore(fill=False)
m_S1 = leafmap.Map(center=(c_lat, c_lon), zoom=5)
m_S1.add_gdf(dswx_gdf_SynEvent, layer_name="DSWX Syn-Event", style=style)
m_S1

### Query the OPERA DSWx-S1 dataset for the AOI

In [ ]:
dswx_results_PreEvent, dswx_gdf_PreEvent = leafmap.nasa_data_search(
    short_name='OPERA_L3_DSWX-S1_V1',
    cloud_hosted=True,
    bounding_box= AOI,
    temporal=(StartDate_PreEvent, EndDate_PreEvent),
    count=-1,  # use -1 to return all datasets
    return_gdf=True,
)

dswx_results_SynEvent, dswx_gdf_SynEvent = leafmap.nasa_data_search(
    short_name='OPERA_L3_DSWX-S1_V1',
    cloud_hosted=True,
    bounding_box= AOI,
    temporal=(StartDate_SynEvent, EndDate_SynEvent),
    count=-1,  # use -1 to return all datasets
    return_gdf=True,
)

In [ ]:
dswx_results_PreEvent[0] #Note this just shows a single MGRS/HLS tile

In [ ]:
dswx_results_SynEvent[0] #Note this just shows a single MGRS/HLS tile

In [ ]:
dswx_gdf_PreEvent.head()